# Customer Lifetime Value (LTV) Prediction Model

This notebook implements a complete LTV prediction pipeline using XGBoost regression with proper validation metrics.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.linear_model import LinearRegression

print("Libraries imported successfully!")

In [ ]:
# Load and prepare data
try:
    # Try to load existing RFM features
    rfm_data = pd.read_csv('../notebooks/rfm_features.csv')
    print(f"Loaded RFM features: {rfm_data.shape}")
except:
    # If RFM features don't exist, create them from raw data
    print("Creating RFM features from raw data...")
    
    # Load raw data
    retail_data = pd.read_csv('../Data/OnlineRetail.csv', encoding='ISO-8859-1', parse_dates=['InvoiceDate'])
    
    # Basic preprocessing
    retail_data['TotalAmount'] = retail_data['Quantity'] * retail_data['UnitPrice']
    retail_data = retail_data.dropna(subset=['CustomerID'])
    retail_data['CustomerID'] = retail_data['CustomerID'].astype(int)
    retail_data = retail_data[retail_data['UnitPrice'] > 0]
    
    # Reference date for RFM calculation
    reference_date = retail_data['InvoiceDate'].max() + timedelta(days=1)
    
    # Create RFM features
    rfm_data = retail_data.groupby('CustomerID').agg({
        'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
        'InvoiceNo': 'nunique',                                    # Frequency
        'TotalAmount': 'sum'                                       # Monetary
    }).rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalAmount': 'Monetary'})
    
    # Additional features
    # Average Order Value
    aov = retail_data.groupby(['CustomerID', 'InvoiceNo'])['TotalAmount'].sum().groupby('CustomerID').mean().reset_index()
    aov.columns = ['CustomerID', 'AvgOrderValue']
    rfm_data = rfm_data.merge(aov, on='CustomerID')
    
    # Customer tenure
    tenure = retail_data.groupby('CustomerID')['InvoiceDate'].agg(['min', 'max']).reset_index()
    tenure['Tenure'] = (tenure['max'] - tenure['min']).dt.days
    rfm_data = rfm_data.merge(tenure[['CustomerID', 'Tenure']], on='CustomerID')
    
    # Product diversity
    diversity = retail_data.groupby('CustomerID')['StockCode'].nunique().reset_index()
    diversity.columns = ['CustomerID', 'ProductDiversity']
    rfm_data = rfm_data.merge(diversity, on='CustomerID')
    
    rfm_data = rfm_data.reset_index()
    print(f"Created RFM features: {rfm_data.shape}")

# Display basic info
print("\nDataset Info:")
print(rfm_data.info())
print("\nFirst few rows:")
print(rfm_data.head())

In [ ]:
# Feature Engineering and Target Variable Creation

# Create target variable (LTV) - we'll use Monetary as our LTV proxy
# In a real scenario, you might want to predict future value based on historical data
rfm_data['LTV'] = rfm_data['Monetary']

# Create additional derived features
rfm_data['RecencyScore'] = pd.qcut(rfm_data['Recency'], 5, labels=[5,4,3,2,1], duplicates='drop')
rfm_data['FrequencyScore'] = pd.qcut(rfm_data['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')
rfm_data['MonetaryScore'] = pd.qcut(rfm_data['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')

# Convert categorical scores to numeric
rfm_data['RecencyScore'] = pd.to_numeric(rfm_data['RecencyScore'], errors='coerce')
rfm_data['FrequencyScore'] = pd.to_numeric(rfm_data['FrequencyScore'], errors='coerce')
rfm_data['MonetaryScore'] = pd.to_numeric(rfm_data['MonetaryScore'], errors='coerce')

# Fill any NaN values
rfm_data = rfm_data.fillna(0)

# Create composite features
rfm_data['RFM_Score'] = rfm_data['RecencyScore'] + rfm_data['FrequencyScore'] + rfm_data['MonetaryScore']
rfm_data['FrequencyMonetaryRatio'] = rfm_data['Frequency'] / (rfm_data['Monetary'] + 1)
rfm_data['AvgDaysBetweenPurchases'] = rfm_data['Tenure'] / (rfm_data['Frequency'] + 1)

print("Feature engineering completed!")
print(f"Dataset shape: {rfm_data.shape}")
print(f"Features: {list(rfm_data.columns)}")

In [ ]:
# Prepare features for modeling

# Select features for modeling (exclude target and ID columns)
feature_columns = ['Recency', 'Frequency', 'AvgOrderValue', 'Tenure', 'ProductDiversity',
                  'RecencyScore', 'FrequencyScore', 'MonetaryScore', 'RFM_Score',
                  'FrequencyMonetaryRatio', 'AvgDaysBetweenPurchases']

# Ensure all feature columns exist
available_features = [col for col in feature_columns if col in rfm_data.columns]
print(f"Available features: {available_features}")

X = rfm_data[available_features]
y = rfm_data['LTV']

# Remove outliers (optional - using IQR method)
Q1 = y.quantile(0.25)
Q3 = y.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Keep data within bounds
mask = (y >= lower_bound) & (y <= upper_bound)
X = X[mask]
y = y[mask]

print(f"Data shape after outlier removal: {X.shape}")
print(f"Target variable statistics:")
print(y.describe())

In [ ]:
# Split data and scale features

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Scale features for some models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data preprocessing completed!")

In [ ]:
# Model Training and Evaluation

def evaluate_model(model, X_test, y_test, model_name):
    """Evaluate model performance"""
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"\n{model_name} Performance:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R² Score: {r2:.4f}")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'predictions': y_pred}

# Dictionary to store results
results = {}

# 1. Linear Regression (baseline)
print("Training Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
results['Linear Regression'] = evaluate_model(lr_model, X_test_scaled, y_test, 'Linear Regression')

# 2. Random Forest
print("\nTraining Random Forest...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
results['Random Forest'] = evaluate_model(rf_model, X_test, y_test, 'Random Forest')

# 3. XGBoost
print("\nTraining XGBoost...")
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)
results['XGBoost'] = evaluate_model(xgb_model, X_test, y_test, 'XGBoost')

In [ ]:
# Hyperparameter Tuning for Best Model

# Find the best model based on R² score
best_model_name = max(results.keys(), key=lambda k: results[k]['R2'])
print(f"Best performing model: {best_model_name}")

# Hyperparameter tuning for XGBoost (assuming it's competitive)
print("\nPerforming hyperparameter tuning for XGBoost...")

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Use a smaller parameter grid for faster execution
xgb_grid = xgb.XGBRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(
    xgb_grid, 
    param_grid, 
    cv=3, 
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {-grid_search.best_score_:.2f}")

# Evaluate the tuned model
best_xgb_model = grid_search.best_estimator_
results['XGBoost Tuned'] = evaluate_model(best_xgb_model, X_test, y_test, 'XGBoost Tuned')

In [ ]:
# Model Comparison and Visualization

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'MAE': [results[model]['MAE'] for model in results.keys()],
    'RMSE': [results[model]['RMSE'] for model in results.keys()],
    'R2': [results[model]['R2'] for model in results.keys()]
})

print("Model Comparison:")
print(comparison_df.round(4))

# Visualize model performance
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# MAE comparison
axes[0].bar(comparison_df['Model'], comparison_df['MAE'])
axes[0].set_title('Mean Absolute Error (MAE)')
axes[0].set_ylabel('MAE')
axes[0].tick_params(axis='x', rotation=45)

# RMSE comparison
axes[1].bar(comparison_df['Model'], comparison_df['RMSE'])
axes[1].set_title('Root Mean Square Error (RMSE)')
axes[1].set_ylabel('RMSE')
axes[1].tick_params(axis='x', rotation=45)

# R² comparison
axes[2].bar(comparison_df['Model'], comparison_df['R2'])
axes[2].set_title('R² Score')
axes[2].set_ylabel('R² Score')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../notebooks/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Feature Importance Analysis

# Get the best model (assuming XGBoost Tuned is best)
final_model = best_xgb_model

# Feature importance
feature_importance = pd.DataFrame({
    'feature': available_features,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance:")
print(feature_importance)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Feature Importance (XGBoost)')
plt.xlabel('Importance')
plt.tight_layout()
plt.savefig('../notebooks/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Generate Final LTV Predictions

# Make predictions for all customers
all_predictions = final_model.predict(X)

# Create final results dataframe
final_results = rfm_data[['CustomerID']].copy()
final_results['Actual_LTV'] = y.values
final_results['Predicted_LTV'] = all_predictions
final_results['LTV_Difference'] = final_results['Predicted_LTV'] - final_results['Actual_LTV']
final_results['LTV_Accuracy'] = 1 - abs(final_results['LTV_Difference']) / final_results['Actual_LTV']

# Add customer segments based on predicted LTV
final_results['LTV_Segment'] = pd.qcut(
    final_results['Predicted_LTV'], 
    q=5, 
    labels=['Low Value', 'Medium-Low', 'Medium', 'Medium-High', 'High Value'],
    duplicates='drop'
)

# Add RFM features for context
final_results = final_results.merge(
    rfm_data[['CustomerID', 'Recency', 'Frequency', 'Monetary', 'AvgOrderValue']], 
    on='CustomerID'
)

print("Final Results Summary:")
print(final_results.describe())

print("\nLTV Segments Distribution:")
print(final_results['LTV_Segment'].value_counts())

# Save final predictions
final_results.to_csv('../notebooks/final_ltv_predictions.csv', index=False)
print("\nFinal LTV predictions saved to 'final_ltv_predictions.csv'")

In [ ]:
# Final Visualizations

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Actual vs Predicted LTV
axes[0, 0].scatter(final_results['Actual_LTV'], final_results['Predicted_LTV'], alpha=0.6)
axes[0, 0].plot([final_results['Actual_LTV'].min(), final_results['Actual_LTV'].max()], 
                [final_results['Actual_LTV'].min(), final_results['Actual_LTV'].max()], 'r--')
axes[0, 0].set_xlabel('Actual LTV')
axes[0, 0].set_ylabel('Predicted LTV')
axes[0, 0].set_title('Actual vs Predicted LTV')

# 2. LTV Segment Distribution
segment_counts = final_results['LTV_Segment'].value_counts()
axes[0, 1].pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%')
axes[0, 1].set_title('Customer LTV Segments')

# 3. Prediction Accuracy Distribution
axes[1, 0].hist(final_results['LTV_Accuracy'].dropna(), bins=30, alpha=0.7)
axes[1, 0].set_xlabel('Prediction Accuracy')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('LTV Prediction Accuracy Distribution')

# 4. LTV by Segment
sns.boxplot(data=final_results, x='LTV_Segment', y='Predicted_LTV', ax=axes[1, 1])
axes[1, 1].set_title('Predicted LTV by Segment')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../notebooks/final_ltv_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n=== LTV PREDICTION MODEL COMPLETED ===")
print(f"Best Model: {best_model_name}")
print(f"Final Model MAE: {results[list(results.keys())[-1]]['MAE']:.2f}")
print(f"Final Model RMSE: {results[list(results.keys())[-1]]['RMSE']:.2f}")
print(f"Final Model R²: {results[list(results.keys())[-1]]['R2']:.4f}")
print(f"Total Customers Analyzed: {len(final_results)}")
print("\nDeliverables Created:")
print("✓ Trained XGBoost model with hyperparameter tuning")
print("✓ Model validation with MAE, RMSE metrics")
print("✓ Final LTV predictions CSV")
print("✓ Customer segmentation based on predicted LTV")
print("✓ Comprehensive visualizations")